### For the program augmented prompt, add "Here is the script that I will use to verify your output: "INSERT SCRIPT" to the end of the direct prompt.

# Time-Dependent Delivery Network

Design a time-dependent delivery network based on the specifications below.

## Problem Description:
Create a delivery network that schedules deliveries across multiple locations using a fleet of vehicles. The network must account for vehicle capacities, location storage capacities, delivery time windows, dynamic travel times, and vehicle speeds to ensure efficient and timely deliveries.

## Constraints:
1. **Locations:**
   - **Total Locations:** 15, labeled from `L0` to `L14`.
   - **Attributes:**
     - **Storage Capacity:** Each location has a storage capacity specified in kilograms (kg). Example: `L0` has a capacity of 500 kg.
     - **Time Window:** Each location has a delivery time window represented as a list of two integers `[start_hour, end_hour]` in 24-hour format. Example: `L3` has a time window of `[9, 11]` corresponding to 09:00-11:00.

2. **Vehicles:**
   - **Total Vehicles:** 7, labeled from `V1` to `V7`.
   - **Attributes:**
     - **Capacity:** Each vehicle has a specific capacity in kilograms (kg). Example: `V1` has a capacity of 100 kg.
     - **Speed:** Each vehicle has a defined speed in kilometers per hour (km/h). Example: `V1` travels at 60 km/h.

3. **Edges (Routes):**
   - **Definition:** Represents travel paths between two distinct locations.
   - **Attributes:**
     - **From:** The starting location ID (e.g., `L0`).
     - **To:** The destination location ID (e.g., `L1`).
     - **Base Travel Time:** The fundamental travel time for the route in minutes.
     - **Hourly Adjustments:** A dictionary where keys are time ranges in the format `"HH-HH"` (24-hour format) and values are additional travel time in minutes applicable during those hours. Example: `{"8-10": 15}` adds 15 minutes to the base travel time between 08:00-10:00.
     - **Maximum Weight Limit:** The maximum weight a vehicle can carry on that route in kilograms (kg).

4. **Operational Constraints:**
   - **Storage Capacity Compliance:** The sum of incoming goods to any location must not exceed its storage capacity.
   - **Vehicle Capacity Compliance:** No vehicle should exceed its capacity on any edge it traverses.
   - **Time Window Compliance:** Departures and arrivals must respect the time windows of locations. Specifically:
     - **Departure Time:** Must be within the `from` location's time window.
     - **Arrival Time:** Must be within the `to` location's time window.
     - **Loading Time:** Assume a fixed loading time of 10 minutes at each location, which must be accounted for when scheduling departures.

## Required Output Format:
Provide the network details in the following Python format. **Strictly adhere to this structure without deviations!**

```python
locations = [
    {"id": "L0", "capacity": int, "time_window": [int, int]},
    {"id": "L1", "capacity": int, "time_window": [int, int]},
    # ... all locations (L2-L14)
]

vehicles = [
    {"id": "V1", "capacity": int, "speed": int},
    {"id": "V2", "capacity": int, "speed": int},
    # ... all vehicles (V3-V7)
]

edges = [
    {"from": "L0", "to": "L1", "base_time": int, "hourly_adjust": {"HH-HH": int}, "max_weight": int},
    {"from": "L1", "to": "L2", "base_time": int, "hourly_adjust": {"HH-HH": int}, "max_weight": int},
    # ... more edges
]


In [2]:
from collections import defaultdict
import re

def verify_time_dependent_delivery_network(locations, vehicles, edges):
    verification_results = {
        "output_structure": {"passed": False, "message": ""},
        "storage_capacity_compliance": {"passed": False, "message": ""},
        "vehicle_capacity_compliance": {"passed": False, "message": ""},
        "time_window_compliance": {"passed": False, "message": ""},
        "overall_passed": False,
        "errors": []
    }
    def add_error(message):
        verification_results["errors"].append(message)

    if not isinstance(locations, list):
        add_error("'locations' should be a list.")
    elif len(locations) != 15:
        add_error(f"Number of locations is {len(locations)}; expected 15.")
    else:
        location_ids = set()
        location_valid = True
        for loc in locations:
            if not isinstance(loc, dict):
                add_error("Each location should be a dictionary.")
                location_valid = False
                continue
            required_loc_keys = {"id", "capacity", "time_window"}
            if not required_loc_keys.issubset(loc.keys()):
                add_error(f"Location {loc.get('id', 'Unknown')} is missing required keys: {required_loc_keys - set(loc.keys())}.")
                location_valid = False
                continue
            if not re.fullmatch(r"L\d+", loc["id"]):
                add_error(f"Location ID '{loc['id']}' is invalid. Expected format 'L0' to 'L14'.")
                location_valid = False
            location_ids.add(loc["id"])
            if not isinstance(loc["capacity"], int) or loc["capacity"] <= 0:
                add_error(f"Location '{loc['id']}' has invalid 'capacity'. Must be a positive integer.")
                location_valid = False
            if (not isinstance(loc["time_window"], list) or
                len(loc["time_window"]) != 2 or
                not all(isinstance(t, int) for t in loc["time_window"])):
                add_error(f"Location '{loc['id']}' has invalid 'time_window'. Must be a list of two integers.")
                location_valid = False
            else:
                start, end = loc["time_window"]
                if not (0 <= start < end <= 24):
                    add_error(f"Location '{loc['id']}' has invalid 'time_window' values: {loc['time_window']}. Must satisfy 0 <= start < end <= 24.")
                    location_valid = False
        if location_valid:
            verification_results["output_structure"]["passed"] = True
        else:
            verification_results["output_structure"]["message"] = "Errors found in 'locations' structure."
    
    if not isinstance(vehicles, list):
        add_error("'vehicles' should be a list.")
    elif len(vehicles) != 7:
        add_error(f"Number of vehicles is {len(vehicles)}; expected 7.")
    else:
        vehicle_ids = set()
        vehicle_valid = True
        vehicle_capacities = []
        for veh in vehicles:
            if not isinstance(veh, dict):
                add_error("Each vehicle should be a dictionary.")
                vehicle_valid = False
                continue
            required_veh_keys = {"id", "capacity", "speed"}
            if not required_veh_keys.issubset(veh.keys()):
                add_error(f"Vehicle {veh.get('id', 'Unknown')} is missing required keys: {required_veh_keys - set(veh.keys())}.")
                vehicle_valid = False
                continue
            if not re.fullmatch(r"V\d+", veh["id"]):
                add_error(f"Vehicle ID '{veh['id']}' is invalid. Expected format 'V1' to 'V7'.")
                vehicle_valid = False
            vehicle_ids.add(veh["id"])
            if not isinstance(veh["capacity"], int) or veh["capacity"] <= 0:
                add_error(f"Vehicle '{veh['id']}' has invalid 'capacity'. Must be a positive integer.")
                vehicle_valid = False
            else:
                vehicle_capacities.append(veh["capacity"])
            if not isinstance(veh["speed"], int) or veh["speed"] <= 0:
                add_error(f"Vehicle '{veh['id']}' has invalid 'speed'. Must be a positive integer.")
                vehicle_valid = False
        if vehicle_valid:
            verification_results["output_structure"]["passed"] = verification_results["output_structure"]["passed"] and True
        else:
            verification_results["output_structure"]["message"] += " Errors found in 'vehicles' structure."
    
    if not isinstance(edges, list):
        add_error("'edges' should be a list.")
    elif len(edges) == 0:
        add_error("No edges defined in the network.")
    else:
        edge_pairs = set()
        incoming_weights = defaultdict(int)
        edge_valid = True
        for edge in edges:
            if not isinstance(edge, dict):
                add_error("Each edge should be a dictionary.")
                edge_valid = False
                continue
            required_edge_keys = {"from", "to", "base_time", "hourly_adjust", "max_weight"}
            if not required_edge_keys.issubset(edge.keys()):
                add_error(f"Edge from '{edge.get('from', 'Unknown')}' to '{edge.get('to', 'Unknown')}' is missing required keys: {required_edge_keys - set(edge.keys())}.")
                edge_valid = False
                continue
            from_id = edge["from"]
            to_id = edge["to"]
            if from_id not in location_ids:
                add_error(f"Edge 'from' location '{from_id}' is not defined among locations.")
                edge_valid = False
            if to_id not in location_ids:
                add_error(f"Edge 'to' location '{to_id}' is not defined among locations.")
                edge_valid = False
            pair = (from_id, to_id)
            if pair in edge_pairs:
                add_error(f"Duplicate edge detected from '{from_id}' to '{to_id}'.")
                edge_valid = False
            else:
                edge_pairs.add(pair)
            if not isinstance(edge["base_time"], int) or edge["base_time"] <= 0:
                add_error(f"Edge from '{from_id}' to '{to_id}' has invalid 'base_time'. Must be a positive integer.")
                edge_valid = False
            if not isinstance(edge["hourly_adjust"], dict):
                add_error(f"Edge from '{from_id}' to '{to_id}' has invalid 'hourly_adjust'. Must be a dictionary.")
                edge_valid = False
            else:
                for time_range, adjustment in edge["hourly_adjust"].items():
                    if not re.fullmatch(r"\d{1,2}-\d{1,2}", time_range):
                        add_error(f"Edge from '{from_id}' to '{to_id}' has invalid time range '{time_range}' in 'hourly_adjust'. Expected format 'HH-HH'.")
                        edge_valid = False
                        continue
                    start, end = map(int, time_range.split('-'))
                    if not (0 <= start < end <= 24):
                        add_error(f"Edge from '{from_id}' to '{to_id}' has invalid time range values '{time_range}'. Must satisfy 0 <= start < end <= 24.")
                        edge_valid = False
                    if not isinstance(adjustment, int) or adjustment < 0:
                        add_error(f"Edge from '{from_id}' to '{to_id}' has invalid adjustment value '{adjustment}' for time range '{time_range}'. Must be a non-negative integer.")
                        edge_valid = False
            if not isinstance(edge["max_weight"], int) or edge["max_weight"] <= 0:
                add_error(f"Edge from '{from_id}' to '{to_id}' has invalid 'max_weight'. Must be a positive integer.")
                edge_valid = False
            else:
                incoming_weights[to_id] += edge["max_weight"]
        if edge_valid:
            verification_results["output_structure"]["passed"] = verification_results["output_structure"]["passed"] and True
        else:
            verification_results["output_structure"]["message"] += " Errors found in 'edges' structure."
    

    storage_compliance_passed = True
    storage_errors = []
    loc_capacity_map = {loc["id"]: loc["capacity"] for loc in locations}
    for loc_id, total_incoming in incoming_weights.items():
        capacity = loc_capacity_map.get(loc_id, 0)
        if total_incoming > capacity:
            storage_compliance_passed = False
            storage_errors.append(f"Total incoming 'max_weight' to location '{loc_id}' is {total_incoming} kg, exceeding its storage capacity of {capacity} kg.")
    if storage_compliance_passed:
        verification_results["storage_capacity_compliance"]["passed"] = True
    else:
        verification_results["storage_capacity_compliance"]["message"] = " ".join(storage_errors)
        add_error("Storage Capacity Compliance Check Failed.")
    
    vehicle_capacity_passed = True
    vehicle_errors = []
    if not vehicle_capacities:
        vehicle_capacity_passed = False
        vehicle_errors.append("No valid vehicle capacities found.")
    else:
        max_vehicle_capacity = max(vehicle_capacities)
        for edge in edges:
            if edge["max_weight"] < max_vehicle_capacity:
                vehicle_capacity_passed = False
                vehicle_errors.append(f"Edge from '{edge['from']}' to '{edge['to']}' has 'max_weight' {edge['max_weight']} kg, which is less than the maximum vehicle capacity of {max_vehicle_capacity} kg.")
    if vehicle_capacity_passed:
        verification_results["vehicle_capacity_compliance"]["passed"] = True
    else:
        verification_results["vehicle_capacity_compliance"]["message"] = " ".join(vehicle_errors)
        add_error("Vehicle Capacity Compliance Check Failed.")
    
    time_window_passed = True
    time_window_errors = []
    loc_time_map = {loc["id"]: loc["time_window"] for loc in locations}
    for edge in edges:
        from_id = edge["from"]
        to_id = edge["to"]
        base_time = edge["base_time"]  
        hourly_adjust = edge["hourly_adjust"] 
        max_adjust = max(hourly_adjust.values()) if hourly_adjust else 0
        total_travel_time = base_time + max_adjust  # in minutes
        total_travel_hours = (total_travel_time + 59) // 60 
        from_tw_start, from_tw_end = loc_time_map[from_id]
        to_tw_start, to_tw_end = loc_time_map[to_id]
        feasible = False
        for dep_hour in range(from_tw_start, from_tw_end):
            dep_time = dep_hour + 0.17 
            arr_time = dep_time + (total_travel_time / 60)
            if to_tw_start <= arr_time <= to_tw_end:
                feasible = True
                break
        if not feasible:
            time_window_passed = False
            time_window_errors.append(f"Edge from '{from_id}' to '{to_id}' with total travel time {total_travel_time} minutes does not fit within the time windows of '{from_id}' [{from_tw_start}-{from_tw_end}] and '{to_id}' [{to_tw_start}-{to_tw_end}].")
    if time_window_passed:
        verification_results["time_window_compliance"]["passed"] = True
    else:
        verification_results["time_window_compliance"]["message"] = " ".join(time_window_errors)
        add_error("Time Window Compliance Check Failed.")
    
    overall_passed = all([
        verification_results["output_structure"]["passed"],
        verification_results["storage_capacity_compliance"]["passed"],
        verification_results["vehicle_capacity_compliance"]["passed"],
        verification_results["time_window_compliance"]["passed"]
    ])
    verification_results["overall_passed"] = overall_passed
    
    return overall_passed, verification_results

#PASTE HERE!

locations = [
    {"id": "L0", "capacity": 500, "time_window": [6, 18]},
    {"id": "L1", "capacity": 300, "time_window": [8, 10]},
    {"id": "L2", "capacity": 450, "time_window": [9, 12]},
    {"id": "L3", "capacity": 600, "time_window": [9, 11]},
    {"id": "L4", "capacity": 800, "time_window": [7, 15]},
    {"id": "L5", "capacity": 350, "time_window": [11, 13]},
    {"id": "L6", "capacity": 700, "time_window": [8, 16]},
    {"id": "L7", "capacity": 400, "time_window": [14, 17]},
    {"id": "L8", "capacity": 550, "time_window": [12, 18]},
    {"id": "L9", "capacity": 900, "time_window": [5, 20]},
    {"id": "L10", "capacity": 200, "time_window": [9, 11]},
    {"id": "L11", "capacity": 750, "time_window": [10, 15]},
    {"id": "L12", "capacity": 1000, "time_window": [6, 20]},
    {"id": "L13", "capacity": 480, "time_window": [7, 12]},
    {"id": "L14", "capacity": 650, "time_window": [13, 16]}
]

vehicles = [
    {"id": "V1", "capacity": 100, "speed": 60},
    {"id": "V2", "capacity": 150, "speed": 55},
    {"id": "V3", "capacity": 200, "speed": 70},
    {"id": "V4", "capacity": 250, "speed": 65},
    {"id": "V5", "capacity": 300, "speed": 75},
    {"id": "V6", "capacity": 180, "speed": 80},
    {"id": "V7", "capacity": 220, "speed": 50}
]

edges = [
    {"from": "L0", "to": "L1", "base_time": 30, "hourly_adjust": {"8-10": 15}, "max_weight": 200},
    {"from": "L0", "to": "L2", "base_time": 45, "hourly_adjust": {"12-14": 10}, "max_weight": 300},
    {"from": "L0", "to": "L3", "base_time": 20, "hourly_adjust": {"9-11": 5}, "max_weight": 250},
    {"from": "L1", "to": "L4", "base_time": 25, "hourly_adjust": {"15-17": 20}, "max_weight": 180},
    {"from": "L1", "to": "L5", "base_time": 35, "hourly_adjust": {"7-9": 10}, "max_weight": 150},
    {"from": "L2", "to": "L6", "base_time": 40, "hourly_adjust": {"10-12": 5}, "max_weight": 220},
    {"from": "L2", "to": "L7", "base_time": 50, "hourly_adjust": {}, "max_weight": 300},
    {"from": "L3", "to": "L8", "base_time": 30, "hourly_adjust": {"14-16": 15}, "max_weight": 200},
    {"from": "L3", "to": "L9", "base_time": 60, "hourly_adjust": {"17-19": 30}, "max_weight": 150},
    {"from": "L4", "to": "L10", "base_time": 25, "hourly_adjust": {"8-10": 10}, "max_weight": 180},
    {"from": "L4", "to": "L11", "base_time": 35, "hourly_adjust": {}, "max_weight": 250},
    {"from": "L5", "to": "L12", "base_time": 20, "hourly_adjust": {"12-14": 15}, "max_weight": 300},
    {"from": "L5", "to": "L13", "base_time": 40, "hourly_adjust": {"10-12": 5}, "max_weight": 200},
    {"from": "L6", "to": "L14", "base_time": 30, "hourly_adjust": {"9-11": 10}, "max_weight": 180},
    {"from": "L1", "to": "L0", "base_time": 30, "hourly_adjust": {"8-10": 15}, "max_weight": 200},
    {"from": "L2", "to": "L0", "base_time": 45, "hourly_adjust": {"12-14": 10}, "max_weight": 300}
]

# Execute verification
verification_output = verify_time_dependent_delivery_network(locations, vehicles, edges)
print("Verification Output:", verification_output[1])


Verification Output: {'output_structure': {'passed': True, 'message': ''}, 'storage_capacity_compliance': {'passed': True, 'message': ''}, 'vehicle_capacity_compliance': {'passed': False, 'message': "Edge from 'L0' to 'L1' has 'max_weight' 200 kg, which is less than the maximum vehicle capacity of 300 kg. Edge from 'L0' to 'L3' has 'max_weight' 250 kg, which is less than the maximum vehicle capacity of 300 kg. Edge from 'L1' to 'L4' has 'max_weight' 180 kg, which is less than the maximum vehicle capacity of 300 kg. Edge from 'L1' to 'L5' has 'max_weight' 150 kg, which is less than the maximum vehicle capacity of 300 kg. Edge from 'L2' to 'L6' has 'max_weight' 220 kg, which is less than the maximum vehicle capacity of 300 kg. Edge from 'L3' to 'L8' has 'max_weight' 200 kg, which is less than the maximum vehicle capacity of 300 kg. Edge from 'L3' to 'L9' has 'max_weight' 150 kg, which is less than the maximum vehicle capacity of 300 kg. Edge from 'L4' to 'L10' has 'max_weight' 180 kg, wh

In [ ]:
import os 
import json

models = [
    "GPT4omini", "Claude35Haiku", "Gemini20Flash", "DeepSeekV3", "Llama3370B", 
    "GPT4o", "Claude35Sonnet", "Gemini15Pro", "Llama31405B", "DeepSeekR1", "o3mini", "o1"
]

prompt_types = ["directprompt", "iterativefeedback", "programaugmented"]

problem_type = "timeDependentDeliveryNetwork"


def get_user_input():
    print("\nAvailable Models:")
    for i, model in enumerate(models):
        print(f"{i+1}. {model}")
    model_choice = int(input("Select a model by number: ")) - 1
    model_selected = models[model_choice]

    print("\nAvailable Prompt Types:")
    for i, prompt in enumerate(prompt_types):
        print(f"{i+1}. {prompt}")
    prompt_choice = int(input("Select a prompt type by number: ")) - 1
    prompt_selected = prompt_types[prompt_choice]

    run_number = int(input("\nEnter the run number (1-5): "))

    return run_number, model_selected, prompt_selected

run_number, model_selected, prompt_selected = get_user_input()

results_dir = "results"
os.makedirs(results_dir, exist_ok=True)

json_file_path = os.path.join(results_dir, f"{problem_type}_{model_selected}_{prompt_selected}_run{run_number}_output.json")


verification_json = {
    "locations": locations,
    "vehicles": vehicles,
    "edges": edges,
    "result": verification_output[0],
    "fullOutput": verification_output[1]
}

with open(json_file_path, mode='w', newline='') as json_file:
    json.dump(verification_json, json_file, indent=4)

print("Save is succesful!",json_file_path)


# Directed Social Network with Influence Relationships

Design a directed social network based on the specifications below.

## Problem Description:
Create a social network graph representing influence relationships among users. Each user has specific attributes, and influence connections must adhere to defined constraints to maintain the integrity and intended structure of the network.

## Constraints:
1. **Users:**
   - Total of 20 users labeled from U0 to U19.
   - Each user has a "trust_score" ranging from 0 to 100.
   - Each user belongs to a "category" based on their trust score:
     - "celebrity" (trust_score ≥ 80)
     - "expert" (70 ≤ trust_score < 80)
     - "regular" (trust_score < 50)

2. **Edges (Influence Relationships):**
   - Directed edges where Ux → Uy indicates that Ux influences Uy.
   - **No self-loops:** A user cannot influence themselves.
   - **Category Constraints:**
     - **Celebrities:** Must have at least 5 outgoing edges.
     - **Regular Users:** Cannot influence experts.
   - **Graph Structure:**
     - The graph must be acyclic (no cycles in influence relationships).

### Required Output Format:
Provide the network details in the following Python format. **Strictly adhere to this structure without deviations!**


users = [
    {"id": "U0", "trust_score": int, "category": "celebrity"},
    {"id": "U1", "trust_score": int, "category": "regular"},
    {"id": "U2", "trust_score": int, "category": "expert"},
    # ... all users (U3-U19)
]

edges = [
    {"from": "U0", "to": "U1"},
    {"from: "U0", "to": "U2"},
    {"from": "U2", "to": "U3"},
    # ... more edges
]


In [ ]:
from collections import defaultdict  

def verify_social_network(users, edges):

    validation_results = {
        "valid_no_self_loops": True,
        "valid_celebrity_outgoing": True,
        "valid_regular_to_expert": True,
        "valid_acyclic": True,
        "valid_user_attributes": True,
        "valid_edge_structure": True,
        "valid_total_users": True,
        "valid_categories": True,
        "valid_trust_scores": True,
        "errors": []
    }

    if not isinstance(users, list):
        validation_results["valid_total_users"] = False
        validation_results["errors"].append("'users' should be a list.")
    elif len(users) != 20:
        validation_results["valid_total_users"] = False
        validation_results["errors"].append(f"Number of users is {len(users)}; expected 20.")
    
    expected_user_ids = {f"U{i}" for i in range(20)}
    actual_user_ids = set()
    user_map = {}
    for user in users:
        if not isinstance(user, dict):
            validation_results["valid_user_attributes"] = False
            validation_results["errors"].append("Each user should be a dictionary.")
            continue
        required_keys = {"id", "trust_score", "category"}
        if not required_keys.issubset(user.keys()):
            missing = required_keys - user.keys()
            validation_results["valid_user_attributes"] = False
            validation_results["errors"].append(f"User {user.get('id', 'Unknown')} is missing keys: {missing}.")
            continue
        user_id = user["id"]
        if user_id not in expected_user_ids:
            validation_results["valid_user_attributes"] = False
            validation_results["errors"].append(f"User ID '{user_id}' is invalid. Expected IDs from U0 to U19.")
        actual_user_ids.add(user_id)
        user_map[user_id] = user
        trust_score = user["trust_score"]
        if not isinstance(trust_score, int) or not (0 <= trust_score <= 100):
            validation_results["valid_trust_scores"] = False
            validation_results["errors"].append(f"User '{user_id}' has invalid 'trust_score': {trust_score}. Must be an integer between 0 and 100.")
        category = user["category"]
        if trust_score >= 80 and category != "celebrity":
            validation_results["valid_categories"] = False
            validation_results["errors"].append(f"User '{user_id}' has 'trust_score' {trust_score} but category '{category}'. Should be 'celebrity'.")
        elif 70 <= trust_score < 80 and category != "expert":
            validation_results["valid_categories"] = False
            validation_results["errors"].append(f"User '{user_id}' has 'trust_score' {trust_score} but category '{category}'. Should be 'expert'.")
        elif trust_score < 50 and category != "regular":
            validation_results["valid_categories"] = False
            validation_results["errors"].append(f"User '{user_id}' has 'trust_score' {trust_score} but category '{category}'. Should be 'regular'.")
        elif 50 <= trust_score < 70 and category not in ["expert", "regular"]:
            validation_results["valid_categories"] = False
            validation_results["errors"].append(f"User '{user_id}' has 'trust_score' {trust_score} but category '{category}'. Should be 'expert' or 'regular'.")

    missing_users = expected_user_ids - actual_user_ids
    if missing_users:
        validation_results["valid_total_users"] = False
        validation_results["errors"].append(f"Missing user IDs: {missing_users}.")

    if not isinstance(edges, list):
        validation_results["valid_edge_structure"] = False
        validation_results["errors"].append("'edges' should be a list.")
    else:
        adj = defaultdict(list)
        for edge in edges:
            if not isinstance(edge, dict):
                validation_results["valid_edge_structure"] = False
                validation_results["errors"].append("Each edge should be a dictionary.")
                continue
            required_edge_keys = {"from", "to"}
            if not required_edge_keys.issubset(edge.keys()):
                missing = required_edge_keys - edge.keys()
                validation_results["valid_edge_structure"] = False
                validation_results["errors"].append(f"Edge is missing keys: {missing}.")
                continue
            from_id = edge["from"]
            to_id = edge["to"]
            if from_id not in expected_user_ids:
                validation_results["valid_edge_structure"] = False
                validation_results["errors"].append(f"Edge 'from' user '{from_id}' is not a valid user ID.")
            if to_id not in expected_user_ids:
                validation_results["valid_edge_structure"] = False
                validation_results["errors"].append(f"Edge 'to' user '{to_id}' is not a valid user ID.")
            adj[from_id].append(to_id)

    for edge in edges:
        if edge["from"] == edge["to"]:
            validation_results["valid_no_self_loops"] = False
            validation_results["errors"].append(f"Self-loop detected: {edge['from']} → {edge['to']}.")

    for user in users:
        if user["category"] == "celebrity":
            user_id = user["id"]
            outgoing = len(adj[user_id])
            if outgoing < 5:
                validation_results["valid_celebrity_outgoing"] = False
                validation_results["errors"].append(f"Celebrity '{user_id}' has {outgoing} outgoing edges; expected at least 5.")

    for edge in edges:
        from_user = user_map.get(edge["from"])
        to_user = user_map.get(edge["to"])
        if from_user and to_user:
            if from_user["category"] == "regular" and to_user["trust_score"] >= 70:
                validation_results["valid_regular_to_expert"] = False
                validation_results["errors"].append(f"Regular user '{from_user['id']}' is influencing expert '{to_user['id']}'.")

    visited = set()
    rec_stack = set()
    def dfs(u):
        visited.add(u)
        rec_stack.add(u)
        for v in adj[u]:
            if v not in visited:
                if dfs(v):
                    return True
            elif v in rec_stack:
                return True
        rec_stack.remove(u)
        return False

    has_cycle = False
    for user_id in expected_user_ids:
        if user_id not in visited:
            if dfs(user_id):
                has_cycle = True
                break
    if has_cycle:
        validation_results["valid_acyclic"] = False
        validation_results["errors"].append("Cycle detected in the network; the graph must be acyclic.")
    return(validation_results['valid_no_self_loops'] and
    validation_results['valid_celebrity_outgoing'] and 
    validation_results['valid_regular_to_expert'] and
     validation_results['valid_acyclic'] and
     validation_results['valid_user_attributes'] and
     validation_results['valid_edge_structure'] and
     validation_results['valid_total_users'] and
      validation_results['valid_categories'] and
      validation_results['valid_trust_scores']
      , validation_results)


#PASTE HERE!
users = [
    {"id": "U0", "trust_score": 85, "category": "celebrity"},
    {"id": "U1", "trust_score": 40, "category": "regular"},
    {"id": "U2", "trust_score": 75, "category": "expert"},
    {"id": "U3", "trust_score": 90, "category": "celebrity"},
    {"id": "U4", "trust_score": 35, "category": "regular"},
    {"id": "U5", "trust_score": 73, "category": "expert"},
    {"id": "U6", "trust_score": 20, "category": "regular"},
    {"id": "U7", "trust_score": 82, "category": "celebrity"},
    {"id": "U8", "trust_score": 45, "category": "regular"},
    {"id": "U9", "trust_score": 77, "category": "expert"},
    {"id": "U10", "trust_score": 30, "category": "regular"},
    {"id": "U11", "trust_score": 88, "category": "celebrity"},
    {"id": "U12", "trust_score": 78, "category": "expert"},
    {"id": "U13", "trust_score": 25, "category": "regular"},
    {"id": "U14", "trust_score": 15, "category": "regular"},
    {"id": "U15", "trust_score": 70, "category": "expert"},
    {"id": "U16", "trust_score": 10, "category": "regular"},
    {"id": "U17", "trust_score": 5, "category": "regular"},
    {"id": "U18", "trust_score": 0, "category": "regular"},
    {"id": "U19", "trust_score": 49, "category": "regular"}
]

edges = [
    # Outgoing edges from celebrity U0 (index 0) – at least 5 edges
    {"from": "U0", "to": "U1"},
    {"from": "U0", "to": "U2"},
    {"from": "U0", "to": "U3"},
    {"from": "U0", "to": "U4"},
    {"from": "U0", "to": "U5"},
    
    # Additional edges from expert U2 (index 2)
    {"from": "U2", "to": "U3"},
    {"from": "U2", "to": "U4"},
    
    # Outgoing edges from celebrity U3 (index 3) – at least 5 edges
    {"from": "U3", "to": "U4"},
    {"from": "U3", "to": "U5"},
    {"from": "U3", "to": "U6"},
    {"from": "U3", "to": "U7"},
    {"from": "U3", "to": "U8"},
    
    # Additional edges from expert U5 (index 5)
    {"from": "U5", "to": "U6"},
    {"from": "U5", "to": "U7"},
    
    # Outgoing edges from celebrity U7 (index 7) – at least 5 edges
    {"from": "U7", "to": "U8"},
    {"from": "U7", "to": "U9"},
    {"from": "U7", "to": "U10"},
    {"from": "U7", "to": "U11"},
    {"from": "U7", "to": "U12"},
    
    # An edge from regular U8 (index 8) that respects category constraints
    {"from": "U8", "to": "U10"},
    
    # Additional edges from expert U9 (index 9)
    {"from": "U9", "to": "U10"},
    {"from": "U9", "to": "U13"},
    
    # Outgoing edges from celebrity U11 (index 11) – at least 5 edges
    {"from": "U11", "to": "U12"},
    {"from": "U11", "to": "U13"},
    {"from": "U11", "to": "U14"},
    {"from": "U11", "to": "U15"},
    {"from": "U11", "to": "U16"},
    
    # Additional edge from expert U12 (index 12)
    {"from": "U12", "to": "U13"},
    
    # Additional edge from expert U15 (index 15)
    {"from": "U15", "to": "U16"},
    
    # Edges among regular users to extend acyclicity
    {"from": "U16", "to": "U17"},
    {"from": "U17", "to": "U18"},
    {"from": "U18", "to": "U19"}
]


verification_output = verify_social_network(users,edges)
print("Verification Output:", verification_output)



In [ ]:
import os 
import json

models = [
    "GPT4omini", "Claude35Haiku", "Gemini20Flash", "DeepSeekV3", "Llama3370B", 
    "GPT4o", "Claude35Sonnet", "Gemini15Pro", "Llama31405B", "DeepSeekR1", "o3mini", "o1"
]

prompt_types = ["directprompt", "iterativefeedback", "programaugmented"]

problem_type = "directedSocialNetwork"


def get_user_input():
    print("\nAvailable Models:")
    for i, model in enumerate(models):
        print(f"{i+1}. {model}")
    model_choice = int(input("Select a model by number: ")) - 1
    model_selected = models[model_choice]

    print("\nAvailable Prompt Types:")
    for i, prompt in enumerate(prompt_types):
        print(f"{i+1}. {prompt}")
    prompt_choice = int(input("Select a prompt type by number: ")) - 1
    prompt_selected = prompt_types[prompt_choice]

    run_number = int(input("\nEnter the run number (1-5): "))

    return run_number, model_selected, prompt_selected

run_number, model_selected, prompt_selected = get_user_input()

results_dir = "results"
os.makedirs(results_dir, exist_ok=True)

json_file_path = os.path.join(results_dir, f"{problem_type}_{model_selected}_{prompt_selected}_run{run_number}_output.json")


verification_json = {
    "users":users,
    "edges":edges,
    "result": verification_output[0],
    "fullOutput": verification_output[1]
}

with open(json_file_path, mode='w', newline='') as json_file:
    json.dump(verification_json, json_file, indent=4)

print("Save is succesful!",json_file_path)

# Quantum Circuit Design  

## Problem Description:
Design a quantum circuit consisting of multiple qubits and quantum gates. The circuit must adhere to specific constraints to ensure proper gate operations, circuit efficiency, and overall functionality. The design should incorporate structural elements like depth and a Directed Acyclic Graph (DAG) while simplifying some of the gate-related rules to enhance accessibility.

## Constraints:
1. **Qubits:**
   - **Total Qubits:** 10, labeled from `Q0` to `Q9`.
   - **Initialization:** All qubits must start in the |0⟩ state.

2. **Gates:**
   - **Types of Gates to Include:**
     - **Single-Qubit Gates:** Hadamard (H), Pauli-X (X), Pauli-Z (Z)
     - **Multi-Qubit Gates:** Controlled NOT (CNOT), SWAP
     - **Measurement:** Measure (Measure)
   - **Gate Operations:**
     - Each gate operates on specific qubits at designated times.
     - **CNOT Gates:** Must operate on qubits that are not adjacent (e.g., Q0 and Q2 are valid; Q0 and Q1 are invalid).
     - **SWAP Gates:** Must operate between pairs of qubits that have identical gate sequences up to that point.
     - **Measurements:** Each qubit can be measured only once and must be the last operation on that qubit.
   - **Gate Restrictions:**
     - **Gate Frequency:** No single-qubit gate can be applied more than twice consecutively on the same qubit.

3. **Circuit Structure:**
   - The circuit must be a Directed Acyclic Graph (DAG); no repeated times for the same qubit.
   - **Layered Operations:** Gates at the same time step must operate on disjoint sets of qubits (i.e., no two gates at the same time can act on the same qubit).
   - **Depth Constraint:** The total number of time steps (layers) must not exceed 30.

4. **Operational Constraints:**
   - **Circuit Reversibility:** Measurements must be the final operations on their respective qubits to maintain circuit reversibility.
   - **Gate Optimization:** The circuit should minimize the total number of gates while satisfying all other constraints.
   - **Final State:** After all operations, all qubits must either be measured or returned to the |0⟩ state.

## Required Output Format:

Provide the circuit details in the following Python format. **Strictly adhere to this structure without deviations!**

```python
circuit = [
    {"gate": "H", "qubits": ["Q0"], "time": 1},
    {"gate": "CNOT", "qubits": ["Q0", "Q2"], "time": 2},
    {"gate": "X", "qubits": ["Q2"], "time": 3},
    {"gate": "Measure", "qubits": ["Q0"], "time": 4},
    # ... more gates
]


In [ ]:
import re
from collections import defaultdict

def verify_quantum_circuit(circuit):
    """
    Verifies the Quantum Circuit Design against the specified constraints.

    Args:
        circuit (list): List of gate operation dictionaries.

    Returns:
        tuple: (bool indicating overall validity, detailed validation results dictionary)
    """
    # Initialize the results dictionary
    verification_results = {
        "valid_output_structure": True,
        "valid_gate_types": True,
        "valid_qubits": True,
        "valid_dag": True,
        "valid_gate_precedences": True,
        "valid_cnot_adjacency": True,
        "valid_swap_constraints": True,
        "valid_measurements": True,
        "valid_gate_restrictions": True,
        "layered_operations": True,
        "depth_constraint": True,
        "gate_optimization": True,
        "final_state_compliance": True,
        "errors": []
    }

    valid_qubits = {f"Q{i}" for i in range(10)}

    # Define allowed gates
    single_qubit_gates = {"H", "X", "Z"}
    multi_qubit_gates = {"CNOT", "SWAP"}
    measurement_gate = "Measure"
    allowed_gates = single_qubit_gates.union(multi_qubit_gates).union({measurement_gate})

    # Initialize data structures
    qubit_operations = defaultdict(list)  
    measurement_counts = defaultdict(int) 
    gate_sequences = defaultdict(list)
    time_layers = defaultdict(set)
    max_time_step = 0


    def add_error(message):
        verification_results["errors"].append(message)

    if not isinstance(circuit, list):
        verification_results["valid_output_structure"] = False
        add_error("'circuit' should be a list.")
        return False, verification_results 

    for idx, gate_op in enumerate(circuit):
        if not isinstance(gate_op, dict):
            verification_results["valid_output_structure"] = False
            add_error(f"Gate operation at index {idx} is not a dictionary.")
            continue

        required_keys = {"gate", "qubits", "time"}
        if not required_keys.issubset(gate_op.keys()):
            missing = required_keys - gate_op.keys()
            verification_results["valid_output_structure"] = False
            add_error(f"Gate operation at index {idx} is missing keys: {missing}.")
            continue

        gate = gate_op["gate"]
        qubits = gate_op["qubits"]
        time = gate_op["time"]


        if gate not in allowed_gates:
            verification_results["valid_gate_types"] = False
            add_error(f"Invalid gate type '{gate}' at index {idx}. Allowed gates: {allowed_gates}.")


        if not isinstance(qubits, list) or not all(isinstance(q, str) for q in qubits):
            verification_results["valid_output_structure"] = False
            add_error(f"'qubits' for gate at index {idx} must be a list of strings.")
            continue

        for q in qubits:
            if q not in valid_qubits:
                verification_results["valid_qubits"] = False
                add_error(f"Invalid qubit ID '{q}' in gate at index {idx}. Expected IDs from Q0 to Q9.")


        if not isinstance(time, int) or time < 0:
            verification_results["valid_output_structure"] = False
            add_error(f"Invalid 'time' value '{time}' in gate at index {idx}. Must be a non-negative integer.")
            continue

        if time > max_time_step:
            max_time_step = time


        for q in qubits:
            qubit_operations[q].append((time, gate))
            gate_sequences[q].append(gate)


        for q in qubits:
            if q in time_layers[time]:
                verification_results["layered_operations"] = False
                add_error(f"Multiple gates operate on qubit '{q}' at time {time}. Gates must operate on disjoint qubit sets per time step.")
            time_layers[time].add(q)

        if gate == "CNOT":
            if len(qubits) != 2:
                verification_results["valid_output_structure"] = False
                add_error(f"CNOT gate at index {idx} must operate on exactly two qubits.")
        elif gate == "SWAP":
            if len(qubits) != 2:
                verification_results["valid_output_structure"] = False
                add_error(f"SWAP gate at index {idx} must operate on exactly two qubits.")
        elif gate in single_qubit_gates and gate != "Measure":
            if len(qubits) != 1:
                verification_results["valid_output_structure"] = False
                add_error(f"{gate} gate at index {idx} must operate on exactly one qubit.")
        elif gate == "Measure":
            if len(qubits) != 1:
                verification_results["valid_output_structure"] = False
                add_error(f"Measure gate at index {idx} must operate on exactly one qubit.")


    if not verification_results["valid_output_structure"]:
        return False, verification_results

    for q, ops in qubit_operations.items():
        times = [t for (t, _) in ops]
        if len(times) != len(set(times)):
            verification_results["valid_dag"] = False
            add_error(f"Qubit '{q}' has overlapping operations at the same time.")

    for q, gates in gate_sequences.items():
        previous_gate = None
        consecutive_gate_count = 0
        for idx, gate in enumerate(gates):
            if gate in single_qubit_gates:
                if previous_gate == gate:
                    consecutive_gate_count += 1
                    if consecutive_gate_count > 2:
                        verification_results["valid_gate_restrictions"] = False
                        add_error(f"Qubit '{q}' has more than two consecutive '{gate}' gates.")
                else:
                    consecutive_gate_count = 1
            else:
                consecutive_gate_count = 0
            previous_gate = gate

    for idx, gate_op in enumerate(circuit):
        gate = gate_op["gate"]
        qubits = gate_op["qubits"]
        if gate == "CNOT":
            q1, q2 = qubits
            q1_num = int(q1[1:])
            q2_num = int(q2[1:])
            if abs(q1_num - q2_num) < 2:
                verification_results["valid_cnot_adjacency"] = False
                add_error(f"CNOT gate at index {idx} operates on adjacent qubits '{q1}' and '{q2}'. Must have at least one qubit gap.")

    for idx, gate_op in enumerate(circuit):
        gate = gate_op["gate"]
        qubits = gate_op["qubits"]
        if gate == "SWAP":
            q1, q2 = qubits
            seq_q1 = gate_sequences[q1][:gate_sequences[q1].index(gate)+1]
            seq_q2 = gate_sequences[q2][:gate_sequences[q2].index(gate)+1]
            if seq_q1[:-1] != seq_q2[:-1]:  # Exclude the current SWAP gate
                verification_results["valid_swap_constraints"] = False
                add_error(f"SWAP gate at index {idx} operates on qubits '{q1}' and '{q2}' with differing gate sequences up to this point.")

    for gate_op in circuit:
        gate = gate_op["gate"]
        qubits = gate_op["qubits"]
        if gate == "Measure":
            q = qubits[0]
            measurement_counts[q] += 1

    for q, count in measurement_counts.items():
        if count > 1:
            verification_results["valid_measurements"] = False
            add_error(f"Qubit '{q}' has been measured {count} times; only one measurement allowed.")

    for time, qubits_in_time in time_layers.items():
        qubits_set = set(qubits_in_time)
        if len(qubits_set) != len(qubits_in_time):
            verification_results["layered_operations"] = False
            add_error(f"Multiple gates operate on the same qubit(s) at time {time}.")

    if max_time_step > 30:
        verification_results["depth_constraint"] = False
        add_error(f"Circuit depth is {max_time_step} time steps; exceeds the maximum allowed depth of 30.")

    for q, gates in gate_sequences.items():
        for i in range(1, len(gates)):
            if gates[i] == gates[i-1] and gates[i] in single_qubit_gates:
                verification_results["gate_optimization"] = False
                add_error(f"Redundant consecutive '{gates[i]}' gates on qubit '{q}' at positions {i-1} and {i}.")

    for q, gates in gate_sequences.items():
        if gates:
            last_gate = gates[-1]
            if last_gate != "Measure" and last_gate != "X" and last_gate != "Z" and last_gate != "H":
                verification_results["final_state_compliance"] = False
                add_error(f"Qubit '{q}' does not end with a Measurement or a gate to reset to |0⟩.")
        else:
            # If no gates were applied, qubit remains in |0⟩
            pass

    # Final Validation
    overall_passed = all([
        verification_results["valid_output_structure"],
        verification_results["valid_gate_types"],
        verification_results["valid_qubits"],
        verification_results["valid_dag"],
        verification_results["valid_gate_precedences"],
        verification_results["valid_cnot_adjacency"],
        verification_results["valid_swap_constraints"],
        verification_results["valid_measurements"],
        verification_results["valid_gate_restrictions"],
        verification_results["layered_operations"],
        verification_results["depth_constraint"],
        verification_results["gate_optimization"],
        verification_results["final_state_compliance"]
    ])
    verification_results["overall_passed"] = overall_passed

    return (verification_results["overall_passed"], verification_results)

circuit = [
    {"gate": "H", "qubits": ["Q0"], "time": 1},
    {"gate": "H", "qubits": ["Q2"], "time": 1},
    {"gate": "H", "qubits": ["Q4"], "time": 1},
    {"gate": "H", "qubits": ["Q6"], "time": 1},
    {"gate": "H", "qubits": ["Q8"], "time": 1},
    {"gate": "H", "qubits": ["Q5"], "time": 1},
    
    {"gate": "CNOT", "qubits": ["Q0", "Q2"], "time": 2},
    {"gate": "CNOT", "qubits": ["Q6", "Q8"], "time": 2},
    
    {"gate": "X", "qubits": ["Q2"], "time": 3},
    {"gate": "Z", "qubits": ["Q8"], "time": 3},
    
    {"gate": "SWAP", "qubits": ["Q4", "Q5"], "time": 4},
    
    {"gate": "Measure", "qubits": ["Q0"], "time": 5},
    {"gate": "Measure", "qubits": ["Q2"], "time": 5},
    {"gate": "Measure", "qubits": ["Q4"], "time": 5},
    {"gate": "Measure", "qubits": ["Q5"], "time": 5},
    {"gate": "Measure", "qubits": ["Q6"], "time": 5},
    {"gate": "Measure", "qubits": ["Q8"], "time": 5},
]


# Execute verification
verification_output = verify_quantum_circuit(circuit)
print("Verification Output:", verification_output)




In [ ]:
import os 
import json

models = [
    "GPT4omini", "Claude35Haiku", "Gemini20Flash", "DeepSeekV3", "Llama3370B", 
    "GPT4o", "Claude35Sonnet", "Gemini15Pro", "Llama31405B", "DeepSeekR1", "o3mini", "o1"
]

prompt_types = ["directprompt", "iterativefeedback", "programaugmented"]

problem_type = "quantum"


def get_user_input():
    """ Asks the user for model, prompt type, run number, and problem type """
    print("\nAvailable Models:")
    for i, model in enumerate(models):
        print(f"{i+1}. {model}")
    model_choice = int(input("Select a model by number: ")) - 1
    model_selected = models[model_choice]

    print("\nAvailable Prompt Types:")
    for i, prompt in enumerate(prompt_types):
        print(f"{i+1}. {prompt}")
    prompt_choice = int(input("Select a prompt type by number: ")) - 1
    prompt_selected = prompt_types[prompt_choice]

    run_number = int(input("\nEnter the run number (1-5): "))

    return run_number, model_selected, prompt_selected

run_number, model_selected, prompt_selected = get_user_input()

results_dir = "results"
os.makedirs(results_dir, exist_ok=True)

json_file_path = os.path.join(results_dir, f"{problem_type}_{model_selected}_{prompt_selected}_run{run_number}_output.json")


verification_json = {
    "circuit": circuit,
    "result": verification_output[0],
    "fullOutput": verification_output[1]
}

with open(json_file_path, mode='w', newline='') as json_file:
    json.dump(verification_json, json_file, indent=4)

print("Save is succesful!",json_file_path)

# Gene-Disease Association Network

Design a gene-disease association network based on the specifications below.

## Problem Description:
Create a bipartite network that models the associations between genes and diseases. This network will represent which genes are associated with which diseases, capturing the strength of each association. The network should adhere to defined constraints to ensure biological relevance and structural integrity.

## Constraints:
1. **Nodes:**
   - **Genes:**
     - Total of 20 genes labeled from G0 to G19.
     - Each gene has a "name" and a "function".
   - **Diseases:**
     - Total of 20 diseases labeled from D0 to D19.
     - Each disease has a "name" and a "severity_level" (e.g., "Low", "Medium", "High").
   
2. **Edges (Associations):**
   - Represents the association between a gene and a disease.
   - **Bipartite Constraint:** Associations can only exist between genes and diseases, not within the same set.
   - **Association Strength:** Each association has a "strength" value ranging from 0.0 to 1.0, indicating the confidence of the association.
   
3. **Degree Constraints:**
   - **Genes:**
     - Each gene must be associated with at least 2 and at most 5 diseases.
   - **Diseases:**
     - Each disease must be associated with at least 3 and at most 10 genes.
   
4. **Structural Constraints:**
   - The network must be bipartite; no edges should connect nodes within the same set (i.e., no gene-gene or disease-disease associations).
   - There should be no duplicate edges (i.e., each gene-disease pair is unique).

## Required Output Format:

Provide the network details in the following Python format. **Strictly adhere to this structure without deviations!**

```python
genes = [
    {"id": "G0", "name": "BRCA1", "function": "DNA repair"},
    {"id": "G1", "name": "TP53", "function": "Tumor suppression"},
    # ... all genes (G2-G19)
]

diseases = [
    {"id": "D0", "name": "Breast Cancer", "severity_level": "High"},
    {"id": "D1", "name": "Lung Cancer", "severity_level": "High"},
    # ... all diseases (D2-D19)
]

associations = [
    {"from": "G0", "to": "D0", "strength": 0.85},
    {"from": "G0", "to": "D2", "strength": 0.60},
    {"from": "G1", "to": "D0", "strength": 0.90},
    # ... more associations
]


In [ ]:
from collections import defaultdict

def verify_gene_disease_association_network(genes, diseases, associations):
    verification_results = {
        "defined_counts": {"passed": False, "message": ""},
        "valid_associations": {"passed": False, "message": ""},
        "degree_constraints": {"passed": False, "message": ""},
        "duplicate_associations": {"passed": False, "message": ""},
    }

    error_messages = []

    if len(genes) == 20 and len(diseases) == 20:
        verification_results["defined_counts"]["passed"] = True
    else:
        missing = []
        if len(genes) != 20:
            missing.append(f"genes count is {len(genes)}; expected 20.")
        if len(diseases) != 20:
            missing.append(f"diseases count is {len(diseases)}; expected 20.")
        verification_results["defined_counts"]["message"] = " ".join(missing)
        error_messages.append("Defined Counts Check Failed.")

    gene_map = {gene["id"]: gene for gene in genes}
    disease_map = {disease["id"]: disease for disease in diseases}

    invalid_associations = []
    for assoc in associations:
        from_id = assoc.get("from")
        to_id = assoc.get("to")
        strength = assoc.get("strength")

        if from_id not in gene_map:
            invalid_associations.append(f"Association from undefined gene '{from_id}' to '{to_id}'.")
        
        if to_id not in disease_map:
            invalid_associations.append(f"Association from '{from_id}' to undefined disease '{to_id}'.")
        
        if not isinstance(strength, (int, float)) or not (0.0 <= strength <= 1.0):
            invalid_associations.append(f"Association strength {strength} for '{from_id}' -> '{to_id}' is out of bounds (0.0 - 1.0).")
    
    if not invalid_associations:
        verification_results["valid_associations"]["passed"] = True
    else:
        verification_results["valid_associations"]["message"] = " ".join(invalid_associations)
        error_messages.append("Valid Associations Check Failed.")

    gene_degrees = defaultdict(int)
    disease_degrees = defaultdict(int)
    for assoc in associations:
        gene_id = assoc.get("from")
        disease_id = assoc.get("to")
        gene_degrees[gene_id] += 1
        disease_degrees[disease_id] += 1

    degree_errors = []

    for gene in genes:
        gene_id = gene["id"]
        degree = gene_degrees.get(gene_id, 0)
        if degree < 2 or degree > 5:
            degree_errors.append(f"Gene '{gene_id}' has {degree} associations; expected between 2 and 5.")

    for disease in diseases:
        disease_id = disease["id"]
        degree = disease_degrees.get(disease_id, 0)
        if degree < 3 or degree > 10:
            degree_errors.append(f"Disease '{disease_id}' has {degree} associations; expected between 3 and 10.")

    if not degree_errors:
        verification_results["degree_constraints"]["passed"] = True
    else:
        verification_results["degree_constraints"]["message"] = " ".join(degree_errors)
        error_messages.append("Degree Constraints Check Failed.")

    seen_pairs = set()
    duplicates = []
    for assoc in associations:
        pair = (assoc.get("from"), assoc.get("to"))
        if pair in seen_pairs:
            duplicates.append(f"Duplicate association found between '{pair[0]}' and '{pair[1]}'.")
        else:
            seen_pairs.add(pair)
    
    if not duplicates:
        verification_results["duplicate_associations"]["passed"] = True
    else:
        verification_results["duplicate_associations"]["message"] = " ".join(duplicates)
        error_messages.append("Duplicate Associations Check Failed.")

    final_results = {
        "constraints": verification_results,
        "overall_passed": not error_messages,
        "errors": error_messages
    }

    return (not error_messages, final_results)

# PASTE HERE!
 
genes = [
    {"id": "G0", "name": "BRCA1", "function": "DNA repair"},
    {"id": "G1", "name": "TP53", "function": "Tumor suppression"},
    {"id": "G2", "name": "CFTR", "function": "Chloride channel"},
    {"id": "G3", "name": "HTT", "function": "Huntingtin protein"},
    {"id": "G4", "name": "FBN1", "function": "Fibrillin-1"},
    {"id": "G5", "name": "APOE", "function": "Lipid transport"},
    {"id": "G6", "name": "EGFR", "function": "Cell growth regulation"},
    {"id": "G7", "name": "VHL", "function": "Hypoxia response"},
    {"id": "G8", "name": "SOD1", "function": "Superoxide dismutase"},
    {"id": "G9", "name": "PARK7", "function": "Protein deglycase"},
    {"id": "G10", "name": "DMD", "function": "Dystrophin production"},
    {"id": "G11", "name": "HFE", "function": "Iron homeostasis"},
    {"id": "G12", "name": "COL1A1", "function": "Collagen formation"},
    {"id": "G13", "name": "MYH7", "function": "Cardiac muscle contraction"},
    {"id": "G14", "name": "FMR1", "function": "RNA binding"},
    {"id": "G15", "name": "PAH", "function": "Phenylalanine hydroxylase"},
    {"id": "G16", "name": "GBA", "function": "Glucosylceramidase"},
    {"id": "G17", "name": "HBB", "function": "Hemoglobin subunit"},
    {"id": "G18", "name": "PRNP", "function": "Prion protein"},
    {"id": "G19", "name": "RYR1", "function": "Calcium release channel"},
]

diseases = [
    {"id": "D0", "name": "Breast Cancer", "severity_level": "High"},
    {"id": "D1", "name": "Lung Cancer", "severity_level": "High"},
    {"id": "D2", "name": "Cystic Fibrosis", "severity_level": "Medium"},
    {"id": "D3", "name": "Huntington's Disease", "severity_level": "High"},
    {"id": "D4", "name": "Marfan Syndrome", "severity_level": "Medium"},
    {"id": "D5", "name": "Alzheimer's Disease", "severity_level": "High"},
    {"id": "D6", "name": "Parkinson's Disease", "severity_level": "Medium"},
    {"id": "D7", "name": "Von Hippel-Lindau Syndrome", "severity_level": "High"},
    {"id": "D8", "name": "Amyotrophic Lateral Sclerosis", "severity_level": "High"},
    {"id": "D9", "name": "Duchenne Muscular Dystrophy", "severity_level": "High"},
    {"id": "D10", "name": "Hemochromatosis", "severity_level": "Medium"},
    {"id": "D11", "name": "Osteogenesis Imperfecta", "severity_level": "Medium"},
    {"id": "D12", "name": "Hypertrophic Cardiomyopathy", "severity_level": "High"},
    {"id": "D13", "name": "Fragile X Syndrome", "severity_level": "High"},
    {"id": "D14", "name": "Phenylketonuria", "severity_level": "Medium"},
    {"id": "D15", "name": "Gaucher Disease", "severity_level": "Medium"},
    {"id": "D16", "name": "Sickle Cell Anemia", "severity_level": "High"},
    {"id": "D17", "name": "Creutzfeldt-Jakob Disease", "severity_level": "High"},
    {"id": "D18", "name": "Malignant Hyperthermia", "severity_level": "High"},
    {"id": "D19", "name": "Retinoblastoma", "severity_level": "High"},
]

associations = [
    {"from": "G0", "to": "D0", "strength": 0.85},
    {"from": "G0", "to": "D1", "strength": 0.75},
    {"from": "G0", "to": "D2", "strength": 0.65},
    {"from": "G1", "to": "D1", "strength": 0.85},
    {"from": "G1", "to": "D2", "strength": 0.75},
    {"from": "G1", "to": "D3", "strength": 0.65},
    {"from": "G2", "to": "D2", "strength": 0.85},
    {"from": "G2", "to": "D3", "strength": 0.75},
    {"from": "G2", "to": "D4", "strength": 0.65},
    {"from": "G3", "to": "D3", "strength": 0.85},
    {"from": "G3", "to": "D4", "strength": 0.75},
    {"from": "G3", "to": "D5", "strength": 0.65},
    {"from": "G4", "to": "D4", "strength": 0.85},
    {"from": "G4", "to": "D5", "strength": 0.75},
    {"from": "G4", "to": "D6", "strength": 0.65},
    {"from": "G5", "to": "D5", "strength": 0.85},
    {"from": "G5", "to": "D6", "strength": 0.75},
    {"from": "G5", "to": "D7", "strength": 0.65},
    {"from": "G6", "to": "D6", "strength": 0.85},
    {"from": "G6", "to": "D7", "strength": 0.75},
    {"from": "G6", "to": "D8", "strength": 0.65},
    {"from": "G7", "to": "D7", "strength": 0.85},
    {"from": "G7", "to": "D8", "strength": 0.75},
    {"from": "G7", "to": "D9", "strength": 0.65},
    {"from": "G8", "to": "D8", "strength": 0.85},
    {"from": "G8", "to": "D9", "strength": 0.75},
    {"from": "G8", "to": "D10", "strength": 0.65},
    {"from": "G9", "to": "D9", "strength": 0.85},
    {"from": "G9", "to": "D10", "strength": 0.75},
    {"from": "G9", "to": "D11", "strength": 0.65},
    {"from": "G10", "to": "D10", "strength": 0.85},
    {"from": "G10", "to": "D11", "strength": 0.75},
    {"from": "G10", "to": "D12", "strength": 0.65},
    {"from": "G11", "to": "D11", "strength": 0.85},
    {"from": "G11", "to": "D12", "strength": 0.75},
    {"from": "G11", "to": "D13", "strength": 0.65},
    {"from": "G12", "to": "D12", "strength": 0.85},
    {"from": "G12", "to": "D13", "strength": 0.75},
    {"from": "G12", "to": "D14", "strength": 0.65},
    {"from": "G13", "to": "D13", "strength": 0.85},
    {"from": "G13", "to": "D14", "strength": 0.75},
    {"from": "G13", "to": "D15", "strength": 0.65},
    {"from": "G14", "to": "D14", "strength": 0.85},
    {"from": "G14", "to": "D15", "strength": 0.75},
    {"from": "G14", "to": "D16", "strength": 0.65},
    {"from": "G15", "to": "D15", "strength": 0.85},
    {"from": "G15", "to": "D16", "strength": 0.75},
    {"from": "G15", "to": "D17", "strength": 0.65},
    {"from": "G16", "to": "D16", "strength": 0.85},
    {"from": "G16", "to": "D17", "strength": 0.75},
    {"from": "G16", "to": "D18", "strength": 0.65},
    {"from": "G17", "to": "D17", "strength": 0.85},
    {"from": "G17", "to": "D18", "strength": 0.75},
    {"from": "G17", "to": "D19", "strength": 0.65},
    {"from": "G18", "to": "D18", "strength": 0.85},
    {"from": "G18", "to": "D19", "strength": 0.75},
    {"from": "G18", "to": "D0", "strength": 0.65},
    {"from": "G19", "to": "D19", "strength": 0.85},
    {"from": "G19", "to": "D0", "strength": 0.75},
    {"from": "G19", "to": "D1", "strength": 0.65},
]

verification_output = verify_gene_disease_association_network(genes, diseases, associations)
print("Gene-Disease Association Network is valid:", verification_output)



In [ ]:
import os 
import json

models = [
    "GPT4omini", "Claude35Haiku", "Gemini20Flash", "DeepSeekV3", "Llama3370B", 
    "GPT4o", "Claude35Sonnet", "Gemini15Pro", "Llama31405B", "DeepSeekR1", "o3mini", "o1"
]

prompt_types = ["directprompt", "iterativefeedback", "programaugmented"]

problem_type = "geneAssociation"


def get_user_input():
    """ Asks the user for model, prompt type, run number, and problem type """
    print("\nAvailable Models:")
    for i, model in enumerate(models):
        print(f"{i+1}. {model}")
    model_choice = int(input("Select a model by number: ")) - 1
    model_selected = models[model_choice]

    print("\nAvailable Prompt Types:")
    for i, prompt in enumerate(prompt_types):
        print(f"{i+1}. {prompt}")
    prompt_choice = int(input("Select a prompt type by number: ")) - 1
    prompt_selected = prompt_types[prompt_choice]

    run_number = int(input("\nEnter the run number (1-5): "))

    return run_number, model_selected, prompt_selected

run_number, model_selected, prompt_selected = get_user_input()

results_dir = "results"
os.makedirs(results_dir, exist_ok=True)

json_file_path = os.path.join(results_dir, f"{problem_type}_{model_selected}_{prompt_selected}_run{run_number}_output.json")

verification_json = {
    "genes": genes,
    "diseases": diseases,
    "associations": associations,
    "result": verification_output[0],
    "fullOutput": verification_output[1]
}

with open(json_file_path, mode='w', newline='') as json_file:
    json.dump(verification_json, json_file, indent=4)

print("Save is succesful!",json_file_path)

## Optimal Network Design Challenge (Enhanced Version)

### Problem Description

Design an optimal transportation network represented as a **directed graph** where nodes represent cities and edges represent one-way roads. The network must satisfy the following constraints to ensure efficiency, connectivity, robustness, and cost-effectiveness:

#### 1. Nodes (Cities):
- **Total:** 8, labeled from `C0` to `C7`.
- **Attributes:**
  - **Population:** Number of inhabitants in each city.
    - `C0`: 1,000
    - `C1`: 500
    - `C2`: 750
    - `C3`: 600
    - `C4`: 900
    - `C5`: 400
    - `C6`: 800
    - `C7`: 650

#### 2. Edges (Roads):
- **Definition:** Represents a one-way road from one city to another.
- **Attributes:**
  - **Distance:** Length of the road in kilometers (km). *(Each road must be ≤ 300 km.)*
  - **Construction Cost:** Cost to build the road in thousand dollars ($K).

#### 3. Constraints:
1. **Connectivity:**
   - The network must be **strongly connected**, meaning there is a directed path from any city to every other city.

2. **Road Capacity:**
   - No single road should be longer than **300 km**.

3. **Cost Optimization:**
   - The **total construction cost** of all roads should not exceed **$10,000K**.

4. **Population Accessibility:**
   - Each city must have **at least two incoming roads** to ensure redundancy and accessibility.

5. **Strategic Road Placement:**
   - Cities `C0` and `C7` are major hubs and **must have at least three outgoing roads** each to distribute traffic efficiently.

6. **Avoiding Redundancy:**
   - **No two cities** should have more than **one direct road** connecting them in the same direction.

7. **Minimizing Total Distance:**
   - The **sum of all road distances** should be minimized to ensure efficient transportation.

8. **2-Edge Robustness:**
   - The network must remain strongly connected if **any single road is removed** (i.e. there must be two edge‑disjoint paths between every ordered pair of cities).

9. **Edge-Disjoint Paths Guarantee:**
   - For every pair of distinct cities, there must exist **at least two completely independent (edge‑disjoint) paths** connecting them.

10. **Balanced Outgoing Degree:**
    - Except for the designated hubs (`C0` and `C7`), the difference between the maximum and minimum number of outgoing roads among all cities must not exceed **2**. This prevents “overloaded” junctions.

11. **Path Efficiency Constraint:**
    - For every pair of cities, the shortest route (by total distance) should be less than **500 km** to ensure quick intercity transit.

12. **Cost–Distance Consistency:**
    - For every road, the construction cost (in $K) must be **between 1.0 and 1.5 times its distance (in km)**.  
      *Example:* A road that is 90 km long must have a cost between **90K** and **135K**.

---

### Required Output Format

Provide the network details in the following Python format. **Strictly adhere to this structure without deviations!**

```python
cities = [
    {"id": "C0", "population": 1000},
    {"id": "C1", "population": 500},
    # ... all cities (C2-C7)
]

roads = [
    {"from": "C0", "to": "C1", "distance": float, "cost": float},
    {"from": "C1", "to": "C2", "distance": float, "cost": float},
    # ... all roads
]


In [12]:
from collections import defaultdict, deque

def verify_optimal_network_design(cities, roads):
    verification_results = {
        "output_structure": {"passed": False, "message": ""},
        "connectivity": {"passed": False, "message": ""},
        "road_capacity": {"passed": False, "message": ""},
        "cost_optimization": {"passed": False, "message": ""},
        "population_accessibility": {"passed": False, "message": ""},
        "strategic_road_placement": {"passed": False, "message": ""},
        "redundancy": {"passed": False, "message": ""},
        "overall_passed": False,
        "errors": []
    }
    
    # 1. Verify cities
    if not isinstance(cities, list):
        verification_results["output_structure"]["message"] = "'cities' should be a list."
        return False, verification_results

    city_ids = set()
    city_population = {}
    for city in cities:
        if not isinstance(city, dict):
            verification_results["errors"].append("Each city should be a dictionary.")
            continue
        if "id" not in city or "population" not in city:
            verification_results["errors"].append(f"City {city.get('id', 'Unknown')} is missing 'id' or 'population'.")
            continue
        city_id = city["id"]
        population = city["population"]
        if city_id in city_ids:
            verification_results["errors"].append(f"Duplicate city ID detected: {city_id}.")
            continue
        city_ids.add(city_id)
        city_population[city_id] = population

    # 2. Verify roads
    if not isinstance(roads, list):
        verification_results["output_structure"]["message"] = "'roads' should be a list."
        return False, verification_results
    
    adjacency = defaultdict(list)
    road_set = set()
    total_cost = 0.0
    road_unique = True
    road_capacity_passed = True
    road_capacity_errors = []
    for road in roads:
        if not isinstance(road, dict):
            verification_results["errors"].append("Each road should be a dictionary.")
            continue
        if "from" not in road or "to" not in road or "distance" not in road or "cost" not in road:
            verification_results["errors"].append(f"Road from {road.get('from', 'Unknown')} to {road.get('to', 'Unknown')} is missing required attributes.")
            continue
        from_city = road["from"]
        to_city = road["to"]
        distance = road["distance"]
        cost = road["cost"]
        
        if from_city not in city_ids:
            verification_results["errors"].append(f"Road 'from' city '{from_city}' is not defined.")
        if to_city not in city_ids:
            verification_results["errors"].append(f"Road 'to' city '{to_city}' is not defined.")
        if not isinstance(distance, (int, float)) or distance > 300:
            verification_results["errors"].append(f"Road from '{from_city}' to '{to_city}' has invalid distance {distance} km (must be <= 300 km).")
            road_capacity_passed = False
            road_capacity_errors.append(f"Road from '{from_city}' to '{to_city}' exceeds the maximum allowed distance of 300 km.")
        if not isinstance(cost, (int, float)) or cost <=0:
            verification_results["errors"].append(f"Road from '{from_city}' to '{to_city}' has invalid cost ${cost}K (must be positive).")
        
        road_key = (from_city, to_city)
        if road_key in road_set:
            verification_results["errors"].append(f"Duplicate road detected from '{from_city}' to '{to_city}'.")
            road_unique = False
        else:
            road_set.add(road_key)
        
        adjacency[from_city].append(to_city)
        total_cost += cost

    # Road Capacity Compliance
    if road_capacity_passed:
        verification_results["road_capacity"]["passed"] = True
    else:
        verification_results["road_capacity"]["passed"] = False
        verification_results["road_capacity"]["message"] = "; ".join(road_capacity_errors)
        verification_results["errors"].extend(road_capacity_errors)
    
    # 3. Check total construction cost
    if total_cost <= 10000:
        verification_results["cost_optimization"]["passed"] = True
    else:
        verification_results["cost_optimization"]["passed"] = False
        verification_results["cost_optimization"]["message"] = f"Total construction cost is ${total_cost}K, exceeding the limit of $10,000K."
        verification_results["errors"].append(verification_results["cost_optimization"]["message"])

    # 4. Check connectivity (Strongly Connected)
    # Using Kosaraju's algorithm
    def kosaraju_scc(adjacency, nodes):
        visited = set()
        stack = []
        
        def dfs(u):
            visited.add(u)
            for v in adjacency[u]:
                if v not in visited:
                    dfs(v)
            stack.append(u)
        
        for u in nodes:
            if u not in visited:
                dfs(u)
        
        # Transpose graph
        transposed = defaultdict(list)
        for u in adjacency:
            for v in adjacency[u]:
                transposed[v].append(u)
        
        # Second pass
        visited.clear()
        scc = []
        
        def dfs_transposed(u, component):
            visited.add(u)
            component.append(u)
            for v in transposed[u]:
                if v not in visited:
                    dfs_transposed(v, component)
        
        while stack:
            u = stack.pop()
            if u not in visited:
                component = []
                dfs_transposed(u, component)
                scc.append(component)
        
        return scc
    
    scc = kosaraju_scc(adjacency, city_ids)
    if len(scc) == 1:
        verification_results["connectivity"]["passed"] = True
    else:
        verification_results["connectivity"]["passed"] = False
        verification_results["connectivity"]["message"] = "The network is not strongly connected."
        verification_results["errors"].append(verification_results["connectivity"]["message"])

    # 5. Check population accessibility (at least two incoming roads per city)
    incoming_roads = defaultdict(int)
    for road in roads:
        incoming_roads[road["to"]] += 1
    population_accessible = True
    for city in city_ids:
        if incoming_roads[city] < 2:
            verification_results["errors"].append(f"City '{city}' has only {incoming_roads[city]} incoming road(s); requires at least two.")
            population_accessible = False
    if population_accessible:
        verification_results["population_accessibility"]["passed"] = True
    else:
        verification_results["population_accessibility"]["passed"] = False

    # 6. Check strategic road placement (C0 and C7 have at least three outgoing roads)
    strategic_passed = True
    for critical_city in ["C0", "C7"]:
        outgoing = len(adjacency[critical_city])
        if outgoing < 3:
            verification_results["errors"].append(f"Critical city '{critical_city}' has only {outgoing} outgoing road(s); requires at least three.")
            strategic_passed = False
    if strategic_passed:
        verification_results["strategic_road_placement"]["passed"] = True
    else:
        verification_results["strategic_road_placement"]["passed"] = False

    # 7. Check redundancy (implied by strong connectivity and multiple incoming roads)
    if verification_results["connectivity"]["passed"] and verification_results["population_accessibility"]["passed"]:
        verification_results["redundancy"]["passed"] = True
    else:
        verification_results["redundancy"]["passed"] = False
        verification_results["redundancy"]["message"] = "Insufficient redundancy due to connectivity or population accessibility issues."
        verification_results["errors"].append(verification_results["redundancy"]["message"])

    # 8. Check road uniqueness
    if road_unique:
        verification_results["output_structure"]["passed"] = True
    else:
        verification_results["output_structure"]["passed"] = False
        verification_results["output_structure"]["message"] = "Duplicate roads detected."
    
    # 9. Overall Pass
    overall_passed = all([
        verification_results["output_structure"]["passed"],
        verification_results["connectivity"]["passed"],
        verification_results["road_capacity"]["passed"],
        verification_results["cost_optimization"]["passed"],
        verification_results["population_accessibility"]["passed"],
        verification_results["strategic_road_placement"]["passed"],
        verification_results["redundancy"]["passed"],
    ])
    verification_results["overall_passed"] = overall_passed
    
    return overall_passed, verification_results


cities = [
    {"id": "C0", "population": 1000},
    {"id": "C1", "population": 500},
    {"id": "C2", "population": 750},
    {"id": "C3", "population": 600},
    {"id": "C4", "population": 900},
    {"id": "C5", "population": 400},
    {"id": "C6", "population": 800},
    {"id": "C7", "population": 650},
]

# Design rationale:
# We construct two complete cycles – a “forward cycle” and its reverse – so that every city
# has exactly 2 incoming edges and every ordered pair of cities is connected by two edge‑disjoint paths.
# This ensures strong connectivity and 2‑edge robustness.
#
# The forward cycle is:
#   C0 → C1 → C2 → C3 → C4 → C5 → C6 → C7 → C0
#
# The reverse cycle (each edge going in the opposite direction) is:
#   C1 → C0, C2 → C1, C3 → C2, C4 → C3, C5 → C4, C6 → C5, C7 → C6, C0 → C7
#
# To meet the strategic requirement that the major hubs C0 and C7 have at least three outgoing roads,
# we add one extra outgoing edge from each:
#   - From C0: an extra edge to C3.
#   - From C7: an extra edge to C4.
#
# All roads are assigned a distance of 50 km and a construction cost of 50K dollars.
# This choice satisfies:
#   - Each road is ≤ 300 km.
#   - Cost–distance consistency (50K is exactly 1.0 times 50 km, within the allowed [1.0, 1.5] range).
#   - The overall total cost (18 × 50K = 900K) is far below the 10,000K budget.
#
# Moreover, with each edge being 50 km, any shortest route between two cities (using at most 4 edges
# along the “shorter” direction around the dual cycles) remains well under the 500 km limit.
#
# Finally, non‐hub cities (C1–C6) have exactly 2 outgoing roads (one from the forward cycle and one from the reverse cycle),
# so the difference between the maximum and minimum outgoing roads (excluding hubs C0 and C7) is 0, satisfying the balanced degree constraint.

roads = [
    # Forward cycle edges
    {"from": "C0", "to": "C1", "distance": 50.0, "cost": 50.0},
    {"from": "C1", "to": "C2", "distance": 50.0, "cost": 50.0},
    {"from": "C2", "to": "C3", "distance": 50.0, "cost": 50.0},
    {"from": "C3", "to": "C4", "distance": 50.0, "cost": 50.0},
    {"from": "C4", "to": "C5", "distance": 50.0, "cost": 50.0},
    {"from": "C5", "to": "C6", "distance": 50.0, "cost": 50.0},
    {"from": "C6", "to": "C7", "distance": 50.0, "cost": 50.0},
    {"from": "C7", "to": "C0", "distance": 50.0, "cost": 50.0},

    # Reverse cycle edges
    {"from": "C1", "to": "C0", "distance": 50.0, "cost": 50.0},
    {"from": "C2", "to": "C1", "distance": 50.0, "cost": 50.0},
    {"from": "C3", "to": "C2", "distance": 50.0, "cost": 50.0},
    {"from": "C4", "to": "C3", "distance": 50.0, "cost": 50.0},
    {"from": "C5", "to": "C4", "distance": 50.0, "cost": 50.0},
    {"from": "C6", "to": "C5", "distance": 50.0, "cost": 50.0},
    {"from": "C7", "to": "C6", "distance": 50.0, "cost": 50.0},
    {"from": "C0", "to": "C7", "distance": 50.0, "cost": 50.0},

    # Extra edges to satisfy hub outbound degree requirements
    {"from": "C0", "to": "C3", "distance": 50.0, "cost": 50.0},
    {"from": "C7", "to": "C4", "distance": 50.0, "cost": 50.0},
]


# Execute verification
verification_output = verify_optimal_network_design(
    cities,
    roads
)
print("Verification Output:", verification_output)


Verification Output: (True, {'output_structure': {'passed': True, 'message': ''}, 'connectivity': {'passed': True, 'message': ''}, 'road_capacity': {'passed': True, 'message': ''}, 'cost_optimization': {'passed': True, 'message': ''}, 'population_accessibility': {'passed': True, 'message': ''}, 'strategic_road_placement': {'passed': True, 'message': ''}, 'redundancy': {'passed': True, 'message': ''}, 'overall_passed': True, 'errors': []})


In [14]:
import os 
import json

models = [
    "GPT4omini", "Claude35Haiku", "Gemini20Flash", "DeepSeekV3", "Llama3370B", 
    "GPT4o", "Claude35Sonnet", "Gemini15Pro", "Llama31405B", "DeepSeekR1", "o3mini", "o1"
]

prompt_types = ["directprompt", "iterativefeedback", "programaugmented"]

problem_type = "cities"


def get_user_input():
    """ Asks the user for model, prompt type, run number, and problem type """
    print("\nAvailable Models:")
    for i, model in enumerate(models):
        print(f"{i+1}. {model}")
    model_choice = int(input("Select a model by number: ")) - 1
    model_selected = models[model_choice]

    print("\nAvailable Prompt Types:")
    for i, prompt in enumerate(prompt_types):
        print(f"{i+1}. {prompt}")
    prompt_choice = int(input("Select a prompt type by number: ")) - 1
    prompt_selected = prompt_types[prompt_choice]

    run_number = int(input("\nEnter the run number (1-5): "))

    return run_number, model_selected, prompt_selected

run_number, model_selected, prompt_selected = get_user_input()

results_dir = "results"
os.makedirs(results_dir, exist_ok=True)

json_file_path = os.path.join(results_dir, f"{problem_type}_{model_selected}_{prompt_selected}_run{run_number}_output.json")


verification_json = {
    "cities":cities,
    "result": verification_output[0],
    "fullOutput": verification_output[1]
}

with open(json_file_path, mode='w', newline='') as json_file:
    json.dump(verification_json, json_file, indent=4)

print("Save is succesful!",json_file_path)


Available Models:
1. GPT4omini
2. Claude35Haiku
3. Gemini20Flash
4. DeepSeekV3
5. Llama3370B
6. GPT4o
7. Claude35Sonnet
8. Gemini15Pro
9. Llama31405B
10. DeepSeekR1
11. o3mini
12. o1
Select a model by number: 11

Available Prompt Types:
1. directprompt
2. iterativefeedback
3. programaugmented
Select a prompt type by number: 2

Enter the run number (1-5): 4
Save is succesful! results/cities_o3mini_iterativefeedback_run4_output.json
